# Dash Board

In [1]:
#import necessary packages
import numpy as np
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns 

# plotly 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# pip install yfinance

from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash

In [2]:
# read cleaned data
DPC = pd.read_excel('Stock_DPC.xlsx',index_col=0)
df = pd.read_excel("Stock_Price_Covid_Period.xlsx",parse_dates=['date'])
price=pd.read_excel('Stock_Price_Covid_Period.xlsx',index_col=0)
industry=pd.read_excel('Company_Info_new.xlsx')
company_info=pd.read_excel('Company_Info_new.xlsx', index_col=0)

# for company_summary table
company_info_name=company_info.dropna(subset=['longName'])
all_options = {i: list(company_info_name.loc[company_info_name['industry']==i, ['longName']]['longName'])+['All'] for i in company_info_name['industry'].unique()}
all_options['ALL'] = list(company_info_name['longName'])


# for bar chart
price.drop(columns=['cumulative_total_cases', 'daily_new_cases','cumulative_total_deaths', 'daily_new_deaths'],inplace=True)
precovid=pd.DataFrame(price.iloc[[0]].dropna(axis=1)).T
pre=list(precovid.index)
price=price.loc[:,pre]
price=price.T
price.reset_index()
price['Symbol'] = price.index
price=price.merge(industry,on='Symbol',how='left')
price.set_index(['Symbol'],inplace=True)
a=price[pd.Timestamp(2020,2,19,0)]

# for covid table and graph
dt=pd.read_csv('worldometer_coronavirus_daily_data.csv',parse_dates=['date'])
daf=pd.read_csv('worldometer_coronavirus_summary_data.csv')[['continent','country','population']]
dt=dt.merge(daf,on='country',how='left').fillna(0)
dt['daily_new_cases']=[0 if i<0 else i for i in dt['daily_new_cases']]
dt = dt[dt['country'] == 'USA']

# others
monthly_mark={0: 'Feb 20', 25: 'Mar', 50: 'Apr', 73: 'May', 104: 'Jun', 134: 'Jul', 165: 'Aug',
              196: 'Sep', 226: 'Oct', 257: 'Nov', 287: 'Dec', 318: 'Jan 21', 349: 'Feb',
              377: 'Mar', 408: 'Apr', 438: 'May', 469: 'Jun', 499: 'Jul'}
tick_text=['Communication Equipment', 'Computer Hardware', 'Electronic Components',
           'Information Technology Services', 'Scientific & Technical Instruments',
           'Semiconductors', 'Software—Application', 'Software—Infrastructure',
           'Solar', 'Other']
color_blue='rgba(67, 86, 148, 2)'

In [4]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions=True

def company_summary_table(df):
    return dash_table.DataTable(
        data=df.to_dict('records'),
        columns=[{'id': c, 'name': c} for c in df.columns],
        style_cell={'textAlign': 'left', 'font-family': 'Times New Roman','fontSize': '16px'},
        style_data={ 'color': 'black', 'backgroundColor': 'white'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgba(216, 225, 228, 0.8)',}],
        style_header={'backgroundColor': 'rgba(216, 225, 228, 0.8)', 'color': 'black','textAlign':'center'})

card11 = dbc.Row([
    dbc.Col([html.Div([
        html.Br(),
        dcc.Dropdown(
            id='industries-radio',
            options=[{'label': k, 'value': k} for k in all_options.keys()],
            value='Software—Infrastructure'
        ),
        html.Br(),
        dcc.Dropdown(id='companies-radio')
    ], style={'width':'80%','display':'inline-block'}
    )]),
    
    dbc.Row([
        dbc.Col([
            html.Div([
                dcc.RadioItems(
                    id='yaxis_type',
                    options=[{'label': i, 'value': i} for i in ['Stock Price','Daily Percentage Change' ]],
                    value='Stock Price',labelStyle={'display': 'inline-block','margin-left': '1vw'}
                ),
                dcc.Graph(id='time_graph'),
                dcc.RangeSlider(
                    id='my_range_slider',
                    min=df.index[0],
                    max=df.index[-1],
                    step=1.5,
                    value=[df.index[0], df.index[-1]],
                    marks=monthly_mark
                )])
        ],width=8),

        dbc.Col([
            html.Div([
                html.Label('Company Overview',style={'margin-left': '6vw','font-family':'Times New Roman','fontSize':'20px'}),
                html.Div(id=('display-table'))
            ],style={'display': 'inline-block', 'vertical-align': 'top','margin-top': '3vw','margin-left': '3vw'})
        ],width=3)
    ])
])

division2_1_card = dbc.Card([
    dbc.CardBody([
        dcc.Graph(id="lollipopgraph")
    ])
], className="h-100")

division3_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Number of Companies Returned to Pre-COVID Level", className="card-title"),
            dcc.Graph(id='bargraph')
        ])
)

division4_card = dbc.Row([
    html.H4("COVID-19 Information",style={'margin-top':'6vw','font-family':'Times New Roman'}),
    dbc.Col(
        dcc.Graph(id="graph"), width=8),
    dbc.Col(
        html.Div(id='display-covid-table'),style={'vertical-align': 'middle', 'margin-top':'6vw'},width=3)
], className="h-100")


card_text_period=dbc.Card(
    [dbc.CardHeader("Period Selected"),dbc.CardBody(id="period_selected")], className="h-100",style={"width": "300px","height": "200px","font-family": "Times New Roman" ,"fontSize": "20px", "text-align": "center",'box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'}
                  
)

card_text_TotalCases=dbc.Card(
    [dbc.CardHeader("Total Cumulative Cases"),dbc.CardBody(id="cases")], className="h-100",style={"width": "300px","height": "200px","font-family":"Times New Roman","fontSize": "20px", "text-align": "center",'box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'}

)


app.layout = dbc.Container(
    dbc.Row([
        dbc.Col([
            html.H1('COVID-19 Impact On Technology Sector', 
                    style={'color': '#F4F4F4','font-family':'Times New Roman' ,'fontSize': '50px','padding':20,
                           'margin': '10px auto', 'text-align': 'center', 'max-width': '1200px'}),
            html.P("The COVID-19 outbreak in 2020 has greatly impacted companies in every sector and every industry. However, the impact is not uniform: some companies benefited from and some companies harmed from this global pandemic. The focus of this dashboard is on the technology sector. What this dashboard tries to offer is insight into how different companies and industries in the technology sector are impacted by COVID-19.",
                    style={'color': '#EDEFF0','font-family':'Times New Roman' ,'fontSize': '14px','padding':15, 'margin': 'auto', 'text-align': 'left', 'max-width': '800px'}
                    ),
            dbc.Row([
                dbc.Col(card_text_period,width={'offset':2}),
                dbc.Col(card_text_TotalCases,)
            ], style={'padding':10}),
        ], width=12),
        
        dbc.Row([dbc.Col([
            html.Div(children="Date Selected",style={'font-family':'Times New Roman','fontSize':'20px'},
                 className="menu-title"),
        dcc.DatePickerSingle(id='my-date-picker-single',
        # display_format='MMMM Do, YYYY',
                             min_date_allowed=pd.Timestamp(2020, 2, 20),
                             max_date_allowed=pd.Timestamp(2021, 7, 30),
                             initial_visible_month=pd.Timestamp(2021, 7, 30),
                             date=pd.Timestamp(2021, 7, 30),
                             style = {'text-align': 'center',
'box-shadow': '0 4px 6px 0 rgba(0, 0, 0, 0.18)'}
                            )],style={"height": "40px",'vertical-align': 'top','margin-top': '0vw','margin-left': '1vw'},
        )

        ]),
        
        dbc.Row(dbc.Col([ 
            dbc.Card(
                dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', 
                         children=[
                             dcc.Tab(label='COMPANY', value='tab-1'),
                             dcc.Tab(label='INDUSTRY', value='tab-2')
                         ])
            ),
            html.Div(id='tabs-content-example-graph'),
            dbc.Row([
                dbc.Col(division4_card)
            ] , style={"height": "400px"})
        ],style={'vertical-align': 'top','margin-top':'5vw'}))
    ], justify="center",style={'background-color': '#45466E','height': '300px'}),
    fluid=True
)


@app.callback(
    Output('companies-radio', 'options'),
    Input('industries-radio', 'value'))
def set_companies_options(selected_industry):
    html
    return [{'label':i, 'value': i} for i in all_options[selected_industry]]

@app.callback(
    Output('companies-radio', 'value'),
    Input('companies-radio', 'options'))
def set_companies_value(available_options):
    return available_options[0]['value']

@app.callback(
    Output('time_graph', 'figure'),
    Input('companies-radio', 'value'),
    Input('industries-radio','value'),
    Input('yaxis_type', 'value'),
    [Input('my_range_slider', 'value')])
def update_graph_stock(company_name,industry_name,yaxis_type,my_range_slider):

    df1 = df.loc[my_range_slider[0]:my_range_slider[1],]
    
    allcom=all_options[industry_name][:-1]
    symbol2=company_info[company_info['longName'].isin(allcom)].index
    symbol2=list(symbol2)
    symbol2.append('date')
    symbol3=[]
    for i in symbol2:
        if i in df1.columns:
            symbol3.append(i)
    print(len(symbol3))
    df2=df1[symbol3]
    df2["mean"]=df2[symbol3].mean(axis=1)
    if company_name=='All':
        if yaxis_type=='Stock Price':
            fig = px.line(df2, x='date', y='mean')
            fig.update_traces(line_color=color_blue)
        else:
            df2=df2[['date','mean']]
            rate=[float('nan')]
            for i in range(1,df2.shape[0]):
                rate.append(round((df2.iloc[i,1]-df2.iloc[i-1,1])/df2.iloc[i-1,1],4))
            df2['rate']=rate
            fig = px.bar(df2, x='date', y=rate)
            fig.update_traces(marker_color=color_blue)
    else:
        symbol=company_info[company_info['longName']==company_name].index[0]
        if yaxis_type=='Stock Price':
            fig = px.line(df1, x='date', y=symbol)
            fig.update_traces(line_color=color_blue)
        else:
            df1=df1[['date',symbol]]
            rate=[float('nan')]
            for i in range(1,df1.shape[0]):
                rate.append(round((df1.iloc[i,1]-df1.iloc[i-1,1])/df1.iloc[i-1,1],4))
            df1['rate']=rate
            fig = px.bar(df1, x='date', y=rate)
            fig.update_traces(marker_color=color_blue)
    fig.update_layout(   
    plot_bgcolor='white'
                        )
    
    return fig

@app.callback(
    Output('display-table', 'children'),
    Input('companies-radio', 'value'))
def summary_table(selected_company):
    company_info_table = company_info.loc[company_info['longName']==selected_company].T.set_index(pd.Index(['Name','State','Industry','Market Cap($M)', '','# of Employees'\
                                            ,'Revenue/Share','ROE','5-year Dividend Yield'])).\
                        drop('').reset_index()
    return company_summary_table(company_info_table)

@app.callback(Output('period_selected', 'children'),
        Input("my-date-picker-single", "date")
             )

def update_output(date_value):
    date_value_text=str(date_value).replace('T00:00:00','')
    end_date=pd.Timestamp(date_value)
    period = ''
    if end_date <= pd.Timestamp(2020, 3, 23):
        DPC_selected = DPC[DPC['date']<=pd.Timestamp(2020, 3, 23)]
        period = 'Period 1'
    elif (end_date<=pd.Timestamp(2020, 6, 9)) & (end_date > pd.Timestamp(2020, 3, 23)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2020, 6, 9)) & (DPC['date'] > pd.Timestamp(2020, 3, 23))]
        period = 'Period 2'
    elif (end_date<=pd.Timestamp(2020, 10, 23)) & (end_date > pd.Timestamp(2020, 6, 9)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2020, 10, 23)) & (DPC['date'] > pd.Timestamp(2020, 6, 9))]
        period = 'Period 3'
    elif (end_date<=pd.Timestamp(2021, 5, 20)) & (end_date > pd.Timestamp(2020, 10, 23)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2021, 5, 20)) & (DPC['date'] > pd.Timestamp(2020, 10, 23))]
        period = 'Period 4'
    else:
        DPC_selected = DPC[DPC['date']>pd.Timestamp(2021, 5, 20)]
        period = 'Period 5'

    return period

@app.callback(Output('cases', 'children'),
        Input("my-date-picker-single", "date")
             )

def update_output(date_value):
    end_date = pd.Timestamp(date_value)
    cumulative_new_case=dt[dt['date'] == end_date]['cumulative_total_cases']
    return cumulative_new_case

@app.callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs-example-graph', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return card11
    elif tab == 'tab-2':
        return dbc.Row(
            [
                dbc.Col(dcc.Graph(id='lollipopgraph'),width = 5),
                dbc.Col(dcc.Graph(id='bgraph'),width = 5),
            ],justify="center"
        )


@app.callback(Output('lollipopgraph', 'figure'),
        Input("my-date-picker-single", "date")
             )

def update_output(date_value):
    date_value_text=str(date_value).replace('T00:00:00','')
    end_date=pd.Timestamp(date_value)
    period = ''
    if end_date <= pd.Timestamp(2020, 3, 23):
        DPC_selected = DPC[DPC['date']<=pd.Timestamp(2020, 3, 23)]
        period = 'Period 1'
    elif (end_date<=pd.Timestamp(2020, 6, 9)) & (end_date > pd.Timestamp(2020, 3, 23)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2020, 6, 9)) & (DPC['date'] > pd.Timestamp(2020, 3, 23))]
        period = 'Period 2'
    elif (end_date<=pd.Timestamp(2020, 10, 23)) & (end_date > pd.Timestamp(2020, 6, 9)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2020, 10, 23)) & (DPC['date'] > pd.Timestamp(2020, 6, 9))]
        period = 'Period 3'
    elif (end_date<=pd.Timestamp(2021, 5, 20)) & (end_date > pd.Timestamp(2020, 10, 23)):
        DPC_selected = DPC[(DPC['date']<=pd.Timestamp(2021, 5, 20)) & (DPC['date'] > pd.Timestamp(2020, 10, 23))]
        period = 'Period 4'
    else:
        DPC_selected = DPC[DPC['date']>pd.Timestamp(2021, 5, 20)]
        period = 'Period 5'
        
    industry = DPC_selected.groupby('industry')['DPC%'].mean().sort_values()
    
    industryindex = []
    for i in industry.index:
        industryindex.append(i+'  ')
    
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=industry.values,
        y=industryindex,
        marker=dict(
            color='rgba(67, 86, 148, 2)',
            line=dict(
                color='rgba(67, 86, 148, 2)',
                width=1.2),
        ),
        orientation='h',
        width=0.02,
        showlegend=False,name=''
    ))
    fig.add_trace(go.Scatter(
        x=industry.values,
        y=industryindex,
        mode='markers',
        line_color='rgba(67, 86, 148, 2)',
        marker_size = 15,
        showlegend=False,name=''
    ))
    
    fig.add_vline(x=0, line_width=0.8, line_color="grey")

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',

        )
    fig.update_layout(xaxis_title='Percentage Change (%)',yaxis_title='Industries',font=dict(
        family="Times New Roman",
        size=12,
        color="#7f7f7f"
    ))
    fig.update_layout(title=f'Average Daily Percentage Change for Industries during {period}',title_x=0.5,
                     font=dict(
        family="Times New Roman",
        size=12,
        color="black"
    ))
    fig.update_xaxes(
        showgrid=True,
        ticks="outside",
        tickson="labels",
        ticklen=10
    )
    fig.show()
    return fig

@app.callback(Output('bgraph', 'figure'),
        Input("my-date-picker-single", "date")
             )

def update_output(date_value):
    date_value_text = str(date_value).replace('T00:00:00','')
    end_date=pd.Timestamp(date_value)
 
    DPC['industry_num']=[0 if i=='other' else 1 if i=='Solar' else 2 if i=='Software—Infrastructure' else 3 if i=='Software—Application'                  else 4 if i =='Semiconductors' else 5 if i== 'Scientific & Technical Instruments' else 6 if i=='Information Technology Services'\
                          else 7 if i =='Electronic Components' else 8 if i== 'Computer Hardware' else 9 for i in DPC['industry']]
    DPC_selected = DPC[(DPC['date']<=end_date)&(DPC['DPC%']!=0)]
    median=DPC_selected.groupby('industry_num').median()
    Max=DPC_selected.groupby('industry_num').max()
    Min=DPC_selected.groupby('industry_num').min()
    t=[i for i in range(0,10)]

    fig = go.Figure()
        
    fig.add_trace(go.Scatter(x=Min['DPC%'], 
                             y =Min.index, 
                             mode = 'markers', 
                             marker_symbol = 'line-ns-open',
                             name='',
                             line_color='rgba(67, 86, 148, 2)',
                             marker_size = 12,
                             marker_line_width=2.4,
                             showlegend=False
                            )
                 )
    fig.add_trace(go.Scatter(x=Max['DPC%'], 
                             y =Max.index, 
                             mode = 'markers', 
                             marker_symbol = 'line-ns-open',
                             name='',
                             line_color='rgba(67, 86, 148, 2)',
                             marker_size = 12,
                             marker_line_width=2.4,
                             showlegend=False
                            )
                 )
    
    # Horizontal line shape

    for i in median.index:
        fig.add_shape(dict(type='line',
                    x0 = Min.loc[i,'DPC%'],
                    y0=i,
                    x1 = Max.loc[i,'DPC%'],
                    y1=i,
                    line=dict(color='grey', width=1)),
                     )

    fig.update_shapes(dict(xref='x', yref='y'))
    fig.update_yaxes(tickvals=[i for i in range (10)], ticktext=['Communication Equipment', 'Computer Hardware', 'Electronic Components',
           'Information Technology Services', 'Scientific & Technical Instruments',
           'Semiconductors', 'Software—Application', 'Software—Infrastructure',
           'Solar', 'Other'])
    fig.add_trace(go.Scatter(
        y=median.index, 
                                    x=median['DPC%'],
                                    mode='markers', 
                                    name='Median',
                                    marker_size=12,
                                    line_color='rgba(67, 86, 148, 2)',
        showlegend=False
    ))                            

    fig.update_layout(title=f'Cumulative Industry Percentage Changes until {date_value_text}',
                      xaxis_title="Percentage Change (%)", yaxis_title="Industries", title_x=0.5,font=dict(
        family="Times New Roman",
        size=12,
        color="black"
    ))
 

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',

        )


    fig.update_xaxes(
        showgrid=True,
        ticks="outside",
        tickson="labels",
        ticklen=10
    )
    

    fig.show()

    
    return fig

@app.callback(Output('bargraph', 'figure'),
              Input("my-date-picker-single", "date"))
    
def update_output(value):
    selected = pd.Timestamp(value)
    selected_col = price[selected]
    industry_col = price["industry"]
    data = precovid.merge(selected_col, how='inner', on=precovid.index)
    data.set_index(['key_0'], inplace=True)
    data = data.merge(industry_col, how='inner', on=data.index)
    data.set_index(['key_0'], inplace=True)
    df = data[data.iloc[:, 0] <= data.iloc[:, 1]]
    graphdata = df.groupby('industry')[selected].count()

    fig = px.bar(graphdata, template="simple_white", width=750, height=500)
    fig.update_layout(title="Number of Companies Returned to Pre-Covid Level",
                      xaxis_title="Industry",
                      yaxis_title="Count",
                      font=dict(family="Times New Roman", size=12, color="black"))
    fig.update_traces(texttemplate=graphdata.values, textposition='outside')
    fig.update_traces(marker_color='rgba(67, 86, 148, 2)',showlegend=False)
    fig.show()
    return fig


@app.callback(Output("graph", "figure"),
        Input("my-date-picker-single", "date"),
        )

def update_output(date_value):
    end_date = pd.Timestamp(date_value)
    date = dt[dt['date'] <= end_date]['date']
    new_case=dt[dt['date'] <= end_date]['daily_new_cases']
    cumulative_new_case=dt[dt['date'] <= end_date]['cumulative_total_cases']
    new_death=dt[dt['date'] <= end_date]['daily_new_deaths']
    cumulative_death=dt[dt['date'] <= end_date]['cumulative_total_deaths']


    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=date,
                    y=new_case,
                    name='Daily New Cases',
                    marker_color='rgba(8, 137, 212,2)'
                    ),secondary_y=False,)
    fig.add_trace(go.Scatter(x=date,
                    y=cumulative_new_case,
                    name='Cumulative Total Cases',
                    marker_color='rgba(67, 86, 148, 2)'
                    ),secondary_y=False,)
    fig.add_trace(go.Scatter(x=date,
                    y=new_death,
                    name='Daily New Death',
                    marker_color='darkgrey'
                    ),secondary_y=True,)
    fig.add_trace(go.Scatter(x=date,
                    y=cumulative_death,
                    name='Cumulative Total Death',
                    marker_color='black'
                    ),secondary_y=True,)   
    fig.update_layout(hovermode='x unified',
                    hoverlabel=dict(
                    bgcolor="white",
                    font_size=10,
                    font_family="Times New Roman"
    ))
    date_value_text = str(date_value).replace('T00:00:00','')
    fig.update_layout(
        title={
        'text': f'Daily New Cases, Daily Deaths, Cumulative Total Cases and Cumulative Total Death on {date_value_text}',
        'y':0.9,
        'x':0.2,
        'xanchor': 'left',
        'yanchor': 'top'},
        font=dict(family="Times New Roman",
        size=10),
        xaxis=dict(tickfont_size=9,showline=True,showticklabels=True,linecolor='black',ticks='inside'),
        yaxis=dict(
            title='# of cases'
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)',
            yanchor="bottom",
            xanchor="center"

        ),
        plot_bgcolor='white',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1 # gap between bars of the same location coordinate.
    )
    fig.update_yaxes(
        title_text="# of New Cases", 
        secondary_y=False)
    fig.update_yaxes(
        title_text="# of Death", 
        secondary_y=True)

    return fig


@app.callback(
    Output('display-covid-table', 'children'),
    Input("my-date-picker-single", "date"))

def summary_table(selected_date):
    us_info_table = dt.loc[dt['date']==selected_date][['daily_new_cases', 'cumulative_total_cases', 'daily_new_deaths','cumulative_total_deaths']].\
            T.set_index(pd.Index(['Daily New Cases', 'Cumulative Total Cases', 'Daily New Death','Cumulative Total Death'])).\
                        reset_index()
    return_table = us_info_table.rename(columns= {'index' :'Categoies', dt.loc[dt['date']==selected_date].index[0] : 'Value',})
    return company_summary_table(return_table)


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
